In [421]:
import sys
sys.path.append('/mnt/c/engDev/git_extrnl/pydantic')
%run __init__.py

import pathlib
from pydantic import BaseModel, ValidationError, validator
import ipywidgets as widgets
from typing import Optional, List, Dict, Type, Any, Union
from markdown import markdown
import ipydatagrid as ipg
import pandas as pd

from ipyautoui.constants import DF_MAP
from ipyautoui._utils import obj_from_string, _markdown
from ipyautoui._auto_logic import get_widget_class_strings, ALLOWED_VALUE_TYPES
from ipyautoui._custom_widgets import AutoUiFileChooser, AutoUiFileUpload, AutoUiDataGrid, AutoModelRunName

In [2]:
# data-definitions only. they define the fields that are required to create a ui object. 

class AutoWidget(BaseModel):
    """data container for a AutoWidget. 
    this is a data container for all auto-widgets.
    it inherits pydantic.BaseModel and uses pydantic validation tools to 
    initiate the "widget" and the "autoui_type" based on the value type and kwargs. 
    using the pydantic ".dict()" and ".json()" commands data can be serialised. the "widget"
    parameter is excluded from the "dict()" and "json()" commands; this means that the 
    output contains strings only and can be written to file.
    """
    value: Any # ALLOWED_VALUE_TYPES
    kwargs: Dict = {}
    autoui_type: str = None
    widget: Any = None
    
    @validator('autoui_type', pre=True, always=True)
    def get_autoui_type(cls, v, values):
        if v is not None:
            # user indicated ui widget type
            if v in DF_MAP.widget.tolist():
                # user provided short-hand mapping to widget type
                cls_str = DF_MAP.set_index('widget')['widget_class_string'].to_dict()[v]
            else:
                # user input class string to be used explicitly
                cls_str = v
        else:
            # run auto ui mapper
            value = values['value']
            cls_str = get_widget_class_strings(values['value'], values['kwargs'], df_map=DF_MAP)
        if cls_str[0:6] != "<class":
            raise ValueError('autoui_type must be a class string')
        return cls_str 
    
    @validator('widget', always=True)
    def get_widget(cls, v, values):
        value = values['value']
        kwargs = values['kwargs']
        if value is not None:
            kwargs = {'value':value} | kwargs
        else:
            kwargs = kwargs
        return obj_from_string(values['autoui_type'])(**kwargs)
        
    class Config:
        arbitrary_types_allowed = True
        
class RowBase(BaseModel):
    name: str = 'name'
    label: str = 'label'

class WidgetRowBase(RowBase,AutoWidget):
    class Config:
        arbitrary_types_allowed = True

class AutoUiBase(BaseModel):
    """
    AutoUi data container. a defined data structure that can be written 
    to dict or JSON or used to initiate a AutoUi object.
    """
    rows: List[WidgetRowBase] = []
    path: pathlib.PurePath = None
        
    class Config:
        arbitrary_types_allowed = True
        fields = {
            'path': {
                'exclude': ...,
            }
        }

In [3]:
# classes that inherit the data objects, and are initialised by them. 
# these classes add the "observe" ability

class WidgetRow(WidgetRowBase):
    """
    class that observes the WidgetRowBase and copies the value from the 
    awc.widget.value to the awc.value on change, this ensures that when the object 
    is written to file (and awc.widget is ignored when written to dict or json),
    thus the value of the widget state is saved
    """
    row: widgets.HBox = None  
    widget: Any = None
        
    def __init__(self, aw: WidgetRowBase):
        """
        Args:
            awc: AutoWidgetBase, data object for AutoWidget
        """
        super().__init__(**aw.dict())
        self._init_row()
        self._init_controls()
        
    def _init_row(self):
        self.row = self._get_row()
    
    def _init_controls(self):
        self.widget.observe(self._copy_value, 'value')
        
    def _copy_value(self, onchange):
        self.value = self.widget.value
        
    def _get_row(self):
        name = self.name
        label = self.label
        return widgets.HBox([self.widget, _markdown(value=f'__{name}__, '), _markdown(value=f'_{label}_')], layout={'width':'100%'}) 
        
    def display(self):
        display(self.row)

    def _ipython_display_(self):
        self.display()
        
    class Config:
        arbitrary_types_allowed = True
        fields = {
            'widget': {
                'exclude': ...,
            },
            'row': {
                'exclude': ...,
            }
        }
        
class AutoUi(AutoUiBase):
    """class that observes the AutoUiBase data structure and copies the value from the 
    self.widget.value to the self.value param on change, this ensures that when the object 
    is written to file (note. self.widget, self.ui are ignored when written to dict or json),
    that the value of the widget state is saved. this relies on the widgets that are being
    observed using traitlets and having an observable "value" field.
    
    TODO:
        make it so it only copies the data that has changed from the widgets 
        to the value field on change insted of the data from all widgets.
    """
    ui: widgets.VBox = None
        
    def __init__(self, aui: AutoUiBase):
        super().__init__(**aui.dict())
        self._init_widgets()
        
    def _init_widgets(self):
        self.rows = [WidgetRow(row) for row in self.rows]
        self.ui = widgets.VBox()
        self.ui.children = [w.row for w in self.rows]
    
    def display(self):
        display(self.ui)

    def _ipython_display_(self):
        self.display()
        
    class Config:
        arbitrary_types_allowed = True
        fields = {
            'ui': {
                'exclude': ...,
            }
        }




In [4]:
from ipyautoui.test_autoui_data import di_test_autologic, rows
import datetime
data = AutoUiBase(**{'rows':rows})

In [5]:
if __name__ == "__main__":
    di_test_autologic = {
        'IntSlider': {
            'value': 1,
            'kwargs': {'min':0, 'max':3} 
        },
        'IntText': {
            'value': 1,
            'kwargs': {}
        },
        'IntRangeSlider': {
            'value': (1,4),
            'kwargs': {'min':0, 'max':5}
        },
        'FloatSlider': {
            'value': 1.4,
            'kwargs': {'min':0, 'max':5}
        },
        'FloatText': {
            'value': 1.4,
            'kwargs': {}
        },
        'FloatRangeSlider': {
            'value': (1.5,5.5),
            'kwargs': {'min':0, 'max':5}
        },
        'Checkbox': {
            'value': True,
            'kwargs': {}
        },
        'Dropdown': {
            'value': 'string',
            'kwargs': {'options':['True', False, 'string']}
        },
        'SelectionRangeSlider': {
            'value': (2, 3),
            'kwargs': {'options':list(range(0,6))}
        },
        'SelectMultiple': {
            'value': ['True', False],
            'kwargs': {'options':['True', False, 'string']}
        },
        'Text': {
            'value': 'short text',
            'kwargs': {}
        },
        'Textarea': {
            'value': 'long text' * 10,
            'kwargs': {}
        },
        'Combobox': {
            'value': 'string',
            'kwargs': {'options': ['True', 'string', 'string2'], 'ensure_option_in_kwargs': True}
        },
        'DatePicker': {
            'value': datetime.date(2021, 8, 9),
            'kwargs': {}
        },
        'FileChooser': {
            'value': pathlib.Path('/mnt/c/engDev/git_mf/ipyautoui/ipyautoui/autoui.ipynb'),
            'kwargs': {}
        },
        'FileUpload': { 
            'value': None,
            'kwargs': {},
        },
        'AutoModelRunName': { 
            #'value': None,
            'autoui_type':"<class 'ipyautoui._custom_widgets.AutoModelRunName'>",
            'kwargs': {},
        },
        'DataGrid': {
            'value': pd.DataFrame.from_dict({'test':[0,1],'df':[1,2]}),
            'kwargs': {}
        }
    }
    rows = [v for k, v in di_test_autologic.items()]

In [7]:
if __name__ == "__main__":
    from IPython.display import Markdown
    display(Markdown('## Widgets'))
    # from ipyautoui.test_autoui_data import di_test_autologic, rows
    # from ipyautoui.tests import test_display_WidgetRow_widget, test_display_AutoUi
    #  test_display_WidgetRow_widget(di_test_autologic)
    rows = [v for k, v in di_test_autologic.items()]
    data = AutoUiBase(**{'rows':rows, 'path':pathlib.PurePath('test.aui.json')})
    ui = AutoUi(data)
    display(ui)
    #ui

## Widgets

In [513]:
import typing
from pydantic import BaseModel, Field
from pydantic import (
    BaseModel,
    NegativeFloat,
    NegativeInt,
    PositiveFloat,
    PositiveInt,
    NonNegativeFloat,
    NonNegativeInt,
    NonPositiveFloat,
    NonPositiveInt,
    conbytes,
    condecimal,
    confloat,
    conint,
    conlist,
    conset,
    constr,
    Field,
)
from pydantic.color import Color
from typing import Tuple, List
from enum import Enum
from datetime import datetime, date
from dataclasses import dataclass
import ipyautoui

class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'
    
class NestedObject(BaseModel):
    string1: str = Field(default='adsf', description='a description about my string')
    int_slider1: conint(ge=0,le=3) =2
    int_text1: int = 1

class TestAutoLogic(BaseModel):
    """<br>this is a test UI form to demonstrate how pydantic class can be used to generate an ipywidget input form"""
    string: str = Field(default='adsf', description='a description about my string')
    int_slider: conint(ge=0,le=3) =2
    int_text: int = 1
    int_range_slider: Tuple[int,int] = Field(default=(0,3),ge=0,le=4)   # check
    float_slider: float = Field(default=2.2, ge=0,le=3) 
    float_text: float =2.2
    float_range_slider: Tuple[float,float] = Field(default=(0,2.2), ge=0,le=3.5)
    checkbox: bool = True
    dropdown: Gender = None
    dropdown_simple: str = Field(default ='asd', enum=['asd','asdf'])
    combobox: str = Field(default ='asd', enum=['asd','asdf'], autoui="<class 'ipywidgets.widgets.widget_string.Combobox'>")
    # selection_range_slider
    select_multiple: List[Gender] = Field(default =['male','female']) # TODO: make this work. requires handling the "anyOf" JSON link
    select_multiple_simple: List[str] = Field(default =['male','female'], enum=['male','female', 'other', 'not_given'])
    text: constr(min_length=0, max_length=20) = 'short text'
    text_area: constr(min_length=0, max_length=200)  = 'long text ' * 50
    date_picker: date = date.today()
    color_picker: Color = 'red'
    file_chooser: pathlib.Path = pathlib.Path('.')
    array: typing.List[str] = Field(default=[], max_items=5)
    # file_upload # TODO: how best to implement this? could auto-save to another location...
    # model_run_name # TODO: try and implement this as a test for custom widgets...
    datagrid: str = Field(default=pd.DataFrame.from_dict({'test':[0,1],'df':[1,2]}).to_json(), format="DataFrame")
    nested: NestedObject = Field(default=None)
    
#  -- ATTACH DEFINITIONS TO PROPERTIES ----------------------
def recursive_search(sch, li):
    if len(li) > 1:
        f = li[0]
        li_tmp = li[1:]
        sch_tmp = sch[f]
        return recursive_search(sch_tmp, li_tmp)
    else:
        return sch[li[0]]

def update_property_from_definition(sch, item, key):
    k = list(item.keys())[0]
    v = list(item.values())[0]
    
    li_filt = v[key].split('/')[1:]
    definition = recursive_search(sch, li_filt)

    di_new = {}
    for k_, v_ in item.items():
        di_new[k_] = definition
    
    sch['properties'][k] = di_new[k]
    return sch

def update_property_definitions(sch, key):
    li_definitions = [{k:v} for k,v in sch['properties'].items() if key in v]
    for l in li_definitions:
         sch = update_property_from_definition(sch, l, key)
    return sch
#  ----------------------------------------------------------

#  -- CHANGE JSON-SCHEMA KEYS TO IPYWIDGET KEYS -------------
di_jsonschema_widget_map = {
    'minimum': 'min',
    'maximum': 'max',
    'enum': 'options',
    'default': 'value',
    'description': 'autoui_label'
}

def update_key(key, di_map=di_jsonschema_widget_map):
    if key in di_map.keys():
        return di_map[key]
    else:
        return key
    
def update_keys(di, di_map=di_jsonschema_widget_map):
    return {update_key(k, di_map): v for k, v in di.items()}

def add_description_field(di):
    for k,v in di.items():
        if 'description' not in v:
            v['description'] =''
        t=v['title']
        d=v['description']
        v['description'] = f"<b>{t}</b>, <i>{d}</i>"
    return di

def rename_schema_keys(di, di_map=di_jsonschema_widget_map):
    di = add_description_field(di)
    rename = {k:update_keys(v, di_map) for k, v in di.items()}
    return rename

def call_rename_schema_keys(di, di_map=di_jsonschema_widget_map, rename_keys=True):
    if rename_keys:
        return rename_schema_keys(di, di_map=di_map)
    else:
        return di
#  ----------------------------------------------------------

#  -- HELPER FUNCTIONS --------------------------------------
def get_type(pr, typ='string'):
    return {k:v for k,v in pr.items() if v['type'] ==typ}

def get_format(pr, typ='date'):
    pr = {k:v for k,v in pr.items() if 'format' in v}
    return {k:v for k,v in pr.items() if v['format'] ==typ}

def get_range(pr, typ='integer'):
    array = get_type(pr, typ='array')
    array = {k:v for k,v in array.items() if len(v['items']) ==2}
    tmp = {}
    for k,v in array.items():
        tmp[k] = v
        for i in v['items']:
            if 'minimum' not in i and 'maximum' not in i:
                tmp = {}
    if len(tmp)==0:
        return tmp
    else:
        rng = {k:v for k, v in tmp.items() if v['items'][0]['type'] == typ}
        for k, v in rng.items():
            rng[k]['minimum'] = v['items'][0]['minimum']
            rng[k]['maximum'] = v['items'][0]['maximum']
    return rng

def drop_enums(pr):
    return {k:v for k,v in pr.items() if 'enum' not in v}

def find_enums(pr):
    return {k:v for k,v in pr.items() if 'enum' in v}

def drop_explicit_autoui(pr):
    return {k:v for k,v in pr.items() if 'autoui' not in v}

def find_explicit_autoui(pr):
    return {k:v for k,v in pr.items() if 'autoui' in v}
#  ----------------------------------------------------------

#  -- FILTER FUNCTIONS --------------------------------------
#  -- find relevant inputs from json-schema properties ------
def get_IntText(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    ints = get_type(pr, typ='integer')
    simple_ints = {k:v for k,v in ints.items() if 'minimum' not in v and 'maximum' not in v}
    return call_rename_schema_keys(simple_ints, rename_keys=rename_keys)

def get_IntSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    ints = get_type(pr, typ='integer')
    simple_ints = {k:v for k,v in ints.items() if 'minimum' in v and 'maximum' in v}
    return call_rename_schema_keys(simple_ints, rename_keys=rename_keys)

def get_FloatText(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    floats = get_type(pr, typ='number')
    simple_floats = {k:v for k,v in floats.items() if 'minimum' not in v and 'maximum' not in v}
    return call_rename_schema_keys(simple_floats, rename_keys=rename_keys)

def get_FloatSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    floats = get_type(pr, typ='number')
    simple_floats = {k:v for k,v in floats.items() if 'minimum' in v and 'maximum' in v}
    return call_rename_schema_keys(simple_floats, rename_keys=rename_keys)

def get_Text(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    strings = get_type(pr)
    short_strings = drop_enums(strings)
    #short_strings = {k:v for k,v in strings.items() if 'maxLength' in v and v['maxLength']<200}
    return call_rename_schema_keys(short_strings, rename_keys=rename_keys)

def get_Textarea(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    strings = get_type(pr)
    simple_strings = drop_enums(strings)
    long_strings = {k:v for k,v in strings.items() if 'maxLength' in v and v['maxLength']>=200}
    return call_rename_schema_keys(long_strings, rename_keys=rename_keys)

def get_Dropdown(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    drops = find_enums(pr)
    drops = {k:v for k,v in drops.items() if v['type'] != 'array'}
    return call_rename_schema_keys(drops, rename_keys=rename_keys)

def get_SelectMultiple(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    mult = find_enums(pr)
    mult = {k:v for k,v in mult.items() if v['type'] == 'array'}
    return call_rename_schema_keys(mult, rename_keys=rename_keys)

def get_Checkbox(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_type(pr, typ='boolean'), rename_keys=rename_keys)

def get_DatePicker(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    date = get_type(pr, 'string')
    date = get_format(date)
    for k,v in date.items():
        v['default'] = datetime.strptime(v['default'], "%Y-%m-%d").date()
    return call_rename_schema_keys(date, rename_keys=rename_keys)

def get_FileChooser(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    file = get_type(pr, 'string')
    file = get_format(file, typ='path')
    return call_rename_schema_keys(file, rename_keys=rename_keys)

def get_DataGrid(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    grid = get_type(pr, 'string')
    grid = get_format(grid, typ='DataFrame')
    return call_rename_schema_keys(grid, rename_keys=rename_keys)

def get_ColorPicker(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    color = get_type(pr, 'string')
    color = get_format(color, typ='color')
    return call_rename_schema_keys(color, rename_keys=rename_keys) 

def get_IntRangeSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_range(pr, typ='integer'), rename_keys=rename_keys)  

def get_FloatRangeSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_range(pr, typ='number'), rename_keys=rename_keys)  

def get_AutoOveride(pr, rename_keys=True):
    pr = find_explicit_autoui(pr)
    return call_rename_schema_keys(pr, rename_keys=rename_keys)  
#  ----------------------------------------------------------

#  -- WIDGET MAPPING ----------------------------------------
#  -- uses filter functions to map schema objects to widgets 
def AutoOveride(str_widget_type):
    return obj_from_string(str_widget_type)

@dataclass
class WidgetMapper:
    fn_filt: typing.Callable
    widget: typing.Callable

DI_WIDGETS_MAPPER = {
    'IntText': WidgetMapper(fn_filt=get_IntText, widget=widgets.IntText),
    'IntSlider': WidgetMapper(fn_filt=get_IntSlider, widget=widgets.IntSlider),
    'FloatText': WidgetMapper(fn_filt=get_FloatText, widget=widgets.FloatText),
    'FloatSlider': WidgetMapper(fn_filt=get_FloatSlider, widget=widgets.FloatSlider),
    'Text': WidgetMapper(fn_filt=get_Text, widget=widgets.Text),
    'Textarea': WidgetMapper(fn_filt=get_Textarea, widget=widgets.Textarea),
    'Dropdown':WidgetMapper(fn_filt=get_Dropdown, widget=widgets.Dropdown),
    'SelectMultiple':WidgetMapper(fn_filt=get_SelectMultiple, widget=widgets.SelectMultiple),
    'Checkbox': WidgetMapper(fn_filt=get_Checkbox, widget=widgets.Checkbox),
    'DatePicker': WidgetMapper(fn_filt=get_DatePicker, widget=widgets.DatePicker),
    'AutoUiFileChooser': WidgetMapper(fn_filt=get_FileChooser, widget=ipyautoui._custom_widgets.AutoUiFileChooser),
    'AutoUiDataGrid': WidgetMapper(fn_filt=get_DataGrid, widget=ipyautoui._custom_widgets.AutoUiDataGrid),
    'ColorPicker': WidgetMapper(fn_filt=get_ColorPicker, widget=widgets.ColorPicker),
    'IntRangeSlider': WidgetMapper(fn_filt=get_IntRangeSlider, widget=widgets.IntRangeSlider),
    'FloatRangeSlider': WidgetMapper(fn_filt=get_FloatRangeSlider, widget=widgets.FloatRangeSlider),
    'AutoOveride': WidgetMapper(fn_filt=get_AutoOveride, widget=AutoOveride),
}

def map_to_widget(sch, di_widgets_mapper=DI_WIDGETS_MAPPER):
    pr = sch['properties']
    li_pr = pr.keys()
    di_ = {}
    for k, v in di_widgets_mapper.items():
        di = v.fn_filt(pr)
        for k_, v_ in di.items():
            di_[k_] = v_
            if 'autoui' not in v_:
                di_[k_]['autoui'] = v.widget
            else:
                di_[k_]['autoui'] = v.widget(v_['autoui'])          
    not_matched = set(di_.keys()) ^ set(li_pr) 
    if len(not_matched)>0:
        print('the following UI items from schema not matched to a widget:')
        print(not_matched)
    li_ordered = [l for l in li_pr if l not in not_matched]
    di_ordered = {l:di_[l] for l in li_ordered}        
    return di_ordered


from traitlets import HasTraits, TraitError, Unicode, default, validate, Dict
import functools

def _init_widgets_and_rows(pr):
    
    di_widgets = {k:v['autoui'](**v) for k,v in pr.items()}
    labels = {k:widgets.HTML(v['autoui_label']) for k,v in pr.items()}
    ui_box = widgets.VBox()
    rows = []
    for (k,v), (k2,v2) in zip(di_widgets.items(), labels.items()):
        rows.append(widgets.HBox([v,v2]))              
    ui_box.children = rows
    return ui_box, di_widgets

class AutoUi(HasTraits):
    value = Dict()
    def __init__(self, pydantic_obj: typing.Type[BaseModel], di_widgets_mapper=None):
        self.out = widgets.Output()
        self._set_di_widgets_mapper(di_widgets_mapper)
        self.pydantic_obj = pydantic_obj
        self._init_schema()
        self._init_form()
        self._init_controls()
        
    def _set_di_widgets_mapper(self, di_widgets_mapper):
        if di_widgets_mapper is None:
            self.di_widgets_mapper = DI_WIDGETS_MAPPER
        else:
            self.di_widgets_mapper = {**DI_WIDGETS_MAPPER, **di_widgets_mapper}
            
    def _init_schema(self):
        sch = self.pydantic_obj.schema().copy()
        key = '$ref'
        self.sch = update_property_definitions(sch, key)
        self.pr = map_to_widget(self.sch, di_widgets_mapper=self.di_widgets_mapper)
    
    def _init_form(self):
        self.ui_form = widgets.VBox()
        title = widgets.HTML(f"<big><b>{self.sch['title']}</b></big> - {self.sch['description']}")
        self.ui_box, self.di_widgets = _init_widgets_and_rows(self.pr)
        self.ui_form.children = [title, self.ui_box]
        
    def _init_controls(self):
        [v.observe(functools.partial(self._watch_change, key=k), 'value') for k, v in self.di_widgets.items()];
        
    def _watch_change(self, change, key=None):
        setattr(self.pydantic_obj, key, self.di_widgets[key].value)
        self.value = self.pydantic_obj.dict()
                    
    def display(self):
        with self.out:
            display(self.ui_form)
        display(self.out)
        
    def _ipython_display_(self):
        self.display()
        
test = TestAutoLogic() 
aui = AutoUi(test)
aui

the following UI items from schema not matched to a widget:
{'array', 'nested', 'select_multiple'}


Output()

In [152]:
di_ = {}
for k, v in di_widgets_mapper.items():
    di = v.fn_filt(pr)
    for k_, v_ in di.items():
        print(k)
        di_[k_] = widgets.HBox([v.widget(**v_), widgets.HTML(v_['label'])])
    #di_widget_inputs = update_schema_to_widgets(schema, di_jsonschema_widget_map)
di_

IntText
IntSlider
FloatText
FloatSlider
Text
Text
Text
Text
Text
Text
Text
Textarea
Dropdown


TraitError: Invalid selection: label not found

In [144]:
for k, v in di_.items():
    display(v)

In [17]:
widgets.ColorPicker()

ColorPicker(value='black')

In [98]:
v_

{'title': 'Date Picker',
 'value': '2021-11-26',
 'type': 'string',
 'format': 'date',
 'label': '<b>Date Picker</b>, <i></i>'}

In [96]:
v_

{'title': 'Date Picker',
 'value': '2021-11-26',
 'type': 'string',
 'format': 'date',
 'label': '<b>Date Picker</b>, <i></i>'}

In [90]:
v.fn_filt(pr)

In [59]:
str(widgets.ColorPicker())

"ColorPicker(value='black')"

In [61]:
type(widgets.ColorPicker)

traitlets.traitlets.MetaHasTraits

In [ ]:
class IntText:

In [405]:
from pydantic import BaseModel, Field


class Model(BaseModel):
    a: int = 'dsf'
    b: int = ...
    c: int = Field(...)
    
Model.schema()

{'title': 'Model',
 'type': 'object',
 'properties': {'a': {'title': 'A', 'default': 'dsf', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'},
  'c': {'title': 'C', 'type': 'integer'}},
 'required': ['b', 'c']}

In [436]:
widgets.IntText(value=10)

IntText(value=10)

In [378]:
for k, v in sch['properties'].items():
    print(k,v['type'])

model string
int_slider integer
int_text integer
int_range_slider array
float_slider number
float_text number
float_range_slider array
checkbox boolean


KeyError: 'type'

In [369]:
from jsonschema import validate, RefResolver
resolver = RefResolver(str(sch),str(sch))
resolver

TypeError: 'RefResolver' object is not callable

In [344]:
from enum import Enum
from pydantic import BaseModel, Field


class FooBar(BaseModel):
    count: int
    size: float = None


class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'


class MainModel(BaseModel):
    """
    This is the description of the main model
    """

    foo_bar: FooBar = Field(...)
    gender: Gender = Field(None, alias='Gender')
    snap: int = Field(
        42,
        title='The Snap',
        description='this is the value of snap',
        gt=30,
        lt=50,
    )

    class Config:
        title = 'Main'


# this is equivalent to json.dumps(MainModel.schema(), indent=2):
# print(MainModel.schema_json(indent=2))

In [277]:
houses = ["Eric's house", "Kenny's house", "Kyle's house", "Stan's house"]

# Each function call represents an elf doing his work 
def deliver_presents_recursively(houses):
    # Worker elf doing his work
    if len(houses) == 1:
        house = houses[0]
        print("Delivering presents to", house)

    # Manager elf doing his work
    else:
        mid = len(houses) // 2
        first_half = houses[:mid]
        second_half = houses[mid:]

        # Divides his work among two elves
        deliver_presents_recursively(first_half)
        deliver_presents_recursively(second_half)
        
deliver_presents_recursively(houses)

Delivering presents to Eric's house
Delivering presents to Kenny's house
Delivering presents to Kyle's house
Delivering presents to Stan's house


In [270]:
tmp = {}
def filt(di, li_filt):
    for f in li_filt:
        tmp = filt(di, k)
    filt
        
    return di[k]

for f in li_filt:
    tmp = filt(di, k)

[filt(di, f) ]

KeyError: 'definitions'

In [201]:
pd.DataFrame.from_dict({'test':[0,1],'df':[1,2]}).to_json()

'{"test":{"0":0,"1":1},"df":{"0":1,"1":2}}'

In [177]:
df = pd.DataFrame.from_dict({'test':[0,1],'df':[1,2]})

,test,df
0,0,1
1,1,2


In [165]:
class Gender:
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'

In [169]:
str(TestAutoLogic)

"<class '__main__.TestAutoLogic'>"

In [153]:
TestAutoLogic.__dict__['__fields__']['int_slider']

ModelField(name='int_slider', type=ConstrainedIntValue, required=False, default=2)

In [154]:
TestAutoLogic.schema()['properties'][ 'int_range_slider']

{'title': 'Int Range Slider',
 'default': (0, 3),
 'type': 'array',
 'items': [{'type': 'integer', 'minimum': 0, 'maximum': 4},
  {'type': 'integer', 'minimum': 0, 'maximum': 4}]}

In [155]:
TestAutoLogic.schema()

{'title': 'TestAutoLogic',
 'description': 'aasdfal;ksdfj',
 'type': 'object',
 'properties': {'int_slider': {'title': 'Int Slider',
   'default': 2,
   'minimum': 0,
   'maximum': 3,
   'type': 'integer'},
  'int_text': {'title': 'Int Text',
   'default': 1,
   'exclusiveMinimum': 0,
   'type': 'integer'},
  'int_range_slider': {'title': 'Int Range Slider',
   'default': (0, 3),
   'type': 'array',
   'items': [{'type': 'integer', 'minimum': 0, 'maximum': 4},
    {'type': 'integer', 'minimum': 0, 'maximum': 4}]},
  'float_slider': {'title': 'Float Slider',
   'default': 2.2,
   'minimum': 0,
   'maximum': 3,
   'type': 'number'},
  'float_text': {'title': 'Float Text', 'default': 2.2, 'type': 'number'},
  'float_range_slider': {'title': 'Float Range Slider',
   'default': (0, 2.2),
   'type': 'array',
   'items': [{'type': 'number'}, {'type': 'number'}]},
  'checkbox': {'title': 'Checkbox', 'default': True, 'type': 'boolean'},
  'dropdown': {'default': 'male', 'allOf': [{'$ref': '#/de

In [101]:
pd.DataFrame.from_dict({'test':[0,1],'df':[1,2]})

,test,df
0,0,1
1,1,2


In [98]:
class TestAutoLogic(BaseModel):
    # int_slider: conint(ge=0,le=3) =2
    # int_text: PositiveInt = 1
    # int_range_slider: Tuple[int,int] = (0,3) # check
    float_slider: float =Field(ge=0,le=10) 

## Widgets

In [67]:
from decimal import Decimal

from pydantic import (
    BaseModel,
    NegativeFloat,
    NegativeInt,
    PositiveFloat,
    PositiveInt,
    NonNegativeFloat,
    NonNegativeInt,
    NonPositiveFloat,
    NonPositiveInt,
    conbytes,
    condecimal,
    confloat,
    conint,
    conlist,
    conset,
    constr,
    Field,
)


class Model(BaseModel):
    lower_bytes: conbytes(to_lower=True)
    short_bytes: conbytes(min_length=2, max_length=10)
    strip_bytes: conbytes(strip_whitespace=True)

    lower_str: constr(to_lower=True)
    short_str: constr(min_length=2, max_length=10)
    regex_str: constr(regex=r'^apple (pie|tart|sandwich)$')
    strip_str: constr(strip_whitespace=True)

    big_int: conint(gt=1000, lt=1024)
    mod_int: conint(multiple_of=5)
    pos_int: PositiveInt
    neg_int: NegativeInt
    non_neg_int: NonNegativeInt
    non_pos_int: NonPositiveInt

    big_float: confloat(gt=1000, lt=1024)
    unit_interval: confloat(ge=0, le=1)
    mod_float: confloat(multiple_of=0.5)
    pos_float: PositiveFloat
    neg_float: NegativeFloat
    non_neg_float: NonNegativeFloat
    non_pos_float: NonPositiveFloat

    short_list: conlist(int, min_items=1, max_items=4)
    short_set: conset(int, min_items=1, max_items=4)

    decimal_positive: condecimal(gt=0)
    decimal_negative: condecimal(lt=0)
    decimal_max_digits_and_places: condecimal(max_digits=2, decimal_places=2)
    mod_decimal: condecimal(multiple_of=Decimal('0.25'))

    bigger_int: int = Field(..., gt=10000, description='asdfadfs')
    
    
sch = Model.schema()

In [68]:
sch

{'title': 'Model',
 'type': 'object',
 'properties': {'lower_bytes': {'title': 'Lower Bytes',
   'type': 'string',
   'format': 'binary'},
  'short_bytes': {'title': 'Short Bytes',
   'minLength': 2,
   'maxLength': 10,
   'type': 'string',
   'format': 'binary'},
  'strip_bytes': {'title': 'Strip Bytes',
   'type': 'string',
   'format': 'binary'},
  'lower_str': {'title': 'Lower Str', 'type': 'string'},
  'short_str': {'title': 'Short Str',
   'minLength': 2,
   'maxLength': 10,
   'type': 'string'},
  'regex_str': {'title': 'Regex Str',
   'pattern': '^apple (pie|tart|sandwich)$',
   'type': 'string'},
  'strip_str': {'title': 'Strip Str', 'type': 'string'},
  'big_int': {'title': 'Big Int',
   'exclusiveMinimum': 1000,
   'exclusiveMaximum': 1024,
   'type': 'integer'},
  'mod_int': {'title': 'Mod Int', 'multipleOf': 5, 'type': 'integer'},
  'pos_int': {'title': 'Pos Int', 'exclusiveMinimum': 0, 'type': 'integer'},
  'neg_int': {'title': 'Neg Int', 'exclusiveMaximum': 0, 'type': 'i

In [29]:
display(ui)

In [52]:
types = [v['type'] for k,v, in sch['properties'].items()]
types

['string',
 'string',
 'string',
 'string',
 'string',
 'string',
 'string',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'number',
 'number',
 'number',
 'number',
 'number',
 'number',
 'number',
 'array',
 'array',
 'number',
 'number',
 'number',
 'number',
 'integer']

In [86]:
sch

{'title': 'Model',
 'type': 'object',
 'properties': {'lower_bytes': {'title': 'Lower Bytes',
   'type': 'string',
   'format': 'binary'},
  'short_bytes': {'title': 'Short Bytes',
   'minLength': 2,
   'maxLength': 10,
   'type': 'string',
   'format': 'binary'},
  'strip_bytes': {'title': 'Strip Bytes',
   'type': 'string',
   'format': 'binary'},
  'lower_str': {'title': 'Lower Str', 'type': 'string'},
  'short_str': {'title': 'Short Str',
   'minLength': 2,
   'maxLength': 10,
   'type': 'string'},
  'regex_str': {'title': 'Regex Str',
   'pattern': '^apple (pie|tart|sandwich)$',
   'type': 'string'},
  'strip_str': {'title': 'Strip Str', 'type': 'string'},
  'big_int': {'title': 'Big Int',
   'exclusiveMinimum': 1000,
   'exclusiveMaximum': 1024,
   'type': 'integer'},
  'mod_int': {'title': 'Mod Int', 'multipleOf': 5, 'type': 'integer'},
  'pos_int': {'title': 'Pos Int', 'exclusiveMinimum': 0, 'type': 'integer'},
  'neg_int': {'title': 'Neg Int', 'exclusiveMaximum': 0, 'type': 'i

In [31]:
class PydanticInts(BaseModel):
    big_int: conint(gt=1000, lt=1024)
    mod_int: conint(multiple_of=5)
    pos_int: PositiveInt
    neg_int: NegativeInt
    non_neg_int: NonNegativeInt
    non_pos_int: NonPositiveInt
    table: 
ints_schema = PydanticInts.schema()
ints_schema

{'title': 'PydanticInts',
 'type': 'object',
 'properties': {'big_int': {'title': 'Big Int',
   'exclusiveMinimum': 1000,
   'exclusiveMaximum': 1024,
   'type': 'integer'},
  'mod_int': {'title': 'Mod Int', 'multipleOf': 5, 'type': 'integer'},
  'pos_int': {'title': 'Pos Int', 'exclusiveMinimum': 0, 'type': 'integer'},
  'neg_int': {'title': 'Neg Int', 'exclusiveMaximum': 0, 'type': 'integer'},
  'non_neg_int': {'title': 'Non Neg Int', 'minimum': 0, 'type': 'integer'},
  'non_pos_int': {'title': 'Non Pos Int', 'maximum': 0, 'type': 'integer'}},
 'required': ['big_int',
  'mod_int',
  'pos_int',
  'neg_int',
  'non_neg_int',
  'non_pos_int']}

In [76]:
import pandas as pd
class Pa(BaseModel):
    path: pathlib.Path

df = pd.DataFrame.from_dict({'as':[0,2,]})
p = Pa(path=pathlib.Path())
p.schema()

{'title': 'Pa',
 'type': 'object',
 'properties': {'path': {'title': 'Path', 'type': 'string', 'format': 'path'}},
 'required': ['path']}

In [79]:
import pandas as pd
class Table(BaseModel):
    df: pd.DataFrame
    path: pathlib.Path
    
    class Config:
        arbitrary_types_allowed = True
        
df = pd.DataFrame.from_dict({'as':[0,2,]})
t = Table(df=df,path=pathlib.Path())


In [37]:
widgets.IntText(step=5)

IntText(value=0, step=5)

In [11]:
from pydantic import BaseModel

class TestAutoLogic(BaseModel):
    int_slider: int = 
    
    
    
    'IntSlider': {
        'value': 1,
        'kwargs': {'min':0, 'max':3} 
    },
    'IntText': {
        'value': 1,
        'kwargs': {}
    },
    'IntRangeSlider': {
        'value': (1,4),
        'kwargs': {'min':0, 'max':5}
    },

1

In [39]:
from pydantic import BaseModel, ValidationError
from pydantic.color import Color

c = Color('ff00ff')
print(c.as_named())
#> magenta
print(c.as_hex())
#> #f0f
c2 = Color('green')
print(c2.as_rgb_tuple())
#> (0, 128, 0)
print(c2.original())
#> green
print(repr(Color('hsl(180, 100%, 50%)')))
#> Color('cyan', rgb=(0, 255, 255))


class Model(BaseModel):
    color: Color


print(Model(color='purple'))
#> color=Color('purple', rgb=(128, 0, 128))
try:
    Model(color='hello')
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    color
      value is not a valid color: string not recognised as a valid color
    (type=value_error.color; reason=string not recognised as a valid color)
    """

magenta
#f0f
(0, 128, 0)
green
Color('cyan', rgb=(0, 255, 255))
color=Color('purple', rgb=(128, 0, 128))
1 validation error for Model
color
  value is not a valid color: string not recognised as a valid color (type=value_error.color; reason=string not recognised as a valid color)


In [40]:
Model.schema()

{'title': 'Model',
 'type': 'object',
 'properties': {'color': {'title': 'Color',
   'type': 'string',
   'format': 'color'}},
 'required': ['color']}

In [54]:
table = [
    [
        'None',
        'null',
        '',
        'JSON Schema Core',
        'Same for `type(None)` or `Literal[None]`'
    ],
    [
        'bool',
        'boolean',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'str',
        'string',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'float',
        'number',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'int',
        'integer',
        '',
        'JSON Schema Validation',
        ''
    ],
    [
        'dict',
        'object',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'list',
        'array',
        {'items': {}},
        'JSON Schema Core',
        ''
    ],
    [
        'tuple',
        'array',
        {'items': {}},
        'JSON Schema Core',
        ''
    ],
    [
        'set',
        'array',
        {'items': {}, 'uniqueItems': True},
        'JSON Schema Validation',
        ''
    ],
    [
        'List[str]',
        'array',
        {'items': {'type': 'string'}},
        'JSON Schema Validation',
        'And equivalently for any other sub type, e.g. `List[int]`.'
    ],
    [
        'Tuple[str, int]',
        'array',
        {'items': [{'type': 'string'}, {'type': 'integer'}]},
        'JSON Schema Validation',
        (
            'And equivalently for any other set of subtypes. Note: If using schemas for OpenAPI, '
            'you shouldn\'t use this declaration, as it would not be valid in OpenAPI (although it is '
            'valid in JSON Schema).'
        )
    ],
    [
        'Dict[str, int]',
        'object',
        {'additionalProperties': {'type': 'integer'}},
        'JSON Schema Validation',
        (
            'And equivalently for any other subfields for dicts. Have in mind that although you can use other types as '
            'keys for dicts with Pydantic, only strings are valid keys for JSON, and so, only str is valid as '
            'JSON Schema key types.'
         )
    ],
    [
        'Union[str, int]',
        'anyOf',
        {'anyOf': [{'type': 'string'}, {'type': 'integer'}]},
        'JSON Schema Validation',
        'And equivalently for any other subfields for unions.'
    ],
    [
        'Enum',
        'enum',
        '{"enum": [...]}',
        'JSON Schema Validation',
        'All the literal values in the enum are included in the definition.'
    ],
    [
        'SecretStr',
        'string',
        {'writeOnly': True},
        'JSON Schema Validation',
        ''
    ],
    [
        'SecretBytes',
        'string',
        {'writeOnly': True},
        'JSON Schema Validation',
        ''
    ],
    [
        'EmailStr',
        'string',
        {'format': 'email'},
        'JSON Schema Validation',
        ''
    ],
    [
        'NameEmail',
        'string',
        {'format': 'name-email'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'AnyUrl',
        'string',
        {'format': 'uri'},
        'JSON Schema Validation',
        ''
    ],
    [
        'Pattern',
        'string',
        {'format': 'regex'},
        'JSON Schema Validation',
        ''
    ],
    [
        'bytes',
        'string',
        {'format': 'binary'},
        'OpenAPI',
        ''
    ],
    [
        'Decimal',
        'number',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'UUID1',
        'string',
        {'format': 'uuid1'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'UUID3',
        'string',
        {'format': 'uuid3'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'UUID4',
        'string',
        {'format': 'uuid4'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'UUID5',
        'string',
        {'format': 'uuid5'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'UUID',
        'string',
        {'format': 'uuid'},
        'Pydantic standard "format" extension',
        'Suggested in OpenAPI.'
    ],
    [
        'FilePath',
        'string',
        {'format': 'file-path'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'DirectoryPath',
        'string',
        {'format': 'directory-path'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'Path',
        'string',
        {'format': 'path'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'datetime',
        'string',
        {'format': 'date-time'},
        'JSON Schema Validation',
        ''
    ],
    [
        'date',
        'string',
        {'format': 'date'},
        'JSON Schema Validation',
        ''
    ],
    [
        'time',
        'string',
        {'format': 'time'},
        'JSON Schema Validation',
        ''
    ],
    [
        'timedelta',
        'number',
        {'format': 'time-delta'},
        'Difference in seconds (a `float`), with Pydantic standard "format" extension',
        'Suggested in JSON Schema repository\'s issues by maintainer.'
    ],
    [
        'Json',
        'string',
        {'format': 'json-string'},
        'Pydantic standard "format" extension',
        ''
    ],
    [
        'IPv4Address',
        'string',
        {'format': 'ipv4'},
        'JSON Schema Validation',
        ''
    ],
    [
        'IPv6Address',
        'string',
        {'format': 'ipv6'},
        'JSON Schema Validation',
        ''
    ],
    [
        'IPvAnyAddress',
        'string',
        {'format': 'ipvanyaddress'},
        'Pydantic standard "format" extension',
        'IPv4 or IPv6 address as used in `ipaddress` module',
    ],
    [
        'IPv4Interface',
        'string',
        {'format': 'ipv4interface'},
        'Pydantic standard "format" extension',
        'IPv4 interface as used in `ipaddress` module',
    ],
    [
        'IPv6Interface',
        'string',
        {'format': 'ipv6interface'},
        'Pydantic standard "format" extension',
        'IPv6 interface as used in `ipaddress` module',
    ],
    [
        'IPvAnyInterface',
        'string',
        {'format': 'ipvanyinterface'},
        'Pydantic standard "format" extension',
        'IPv4 or IPv6 interface as used in `ipaddress` module',
    ],
    [
        'IPv4Network',
        'string',
        {'format': 'ipv4network'},
        'Pydantic standard "format" extension',
        'IPv4 network as used in `ipaddress` module',
    ],
    [
        'IPv6Network',
        'string',
        {'format': 'ipv6network'},
        'Pydantic standard "format" extension',
        'IPv6 network as used in `ipaddress` module',
    ],
    [
        'IPvAnyNetwork',
        'string',
        {'format': 'ipvanynetwork'},
        'Pydantic standard "format" extension',
        'IPv4 or IPv6 network as used in `ipaddress` module',
    ],
    [
        'StrictBool',
        'boolean',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'StrictStr',
        'string',
        '',
        'JSON Schema Core',
        ''
    ],
    [
        'ConstrainedStr',
        'string',
        '',
        'JSON Schema Core',
        (
            'If the type has values declared for the constraints, they are included as validations. '
            'See the mapping for `constr` below.'
        )
    ],
    [
        'constr(regex=\'^text$\', min_length=2, max_length=10)',
        'string',
        {'pattern': '^text$', 'minLength': 2, 'maxLength': 10},
        'JSON Schema Validation',
        'Any argument not passed to the function (not defined) will not be included in the schema.'
    ],
    [
        'ConstrainedInt',
        'integer',
        '',
        'JSON Schema Core',
        (
            'If the type has values declared for the constraints, they are included as validations. '
            'See the mapping for `conint` below.'
        )
    ],
    [
        'conint(gt=1, ge=2, lt=6, le=5, multiple_of=2)',
        'integer',
        {'maximum': 5, 'exclusiveMaximum': 6, 'minimum': 2, 'exclusiveMinimum': 1, 'multipleOf': 2},
        '',
        'Any argument not passed to the function (not defined) will not be included in the schema.'
    ],
    [
        'PositiveInt',
        'integer',
        {'exclusiveMinimum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NegativeInt',
        'integer',
        {'exclusiveMaximum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NonNegativeInt',
        'integer',
        {'minimum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NonPositiveInt',
        'integer',
        {'maximum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'ConstrainedFloat',
        'number',
        '',
        'JSON Schema Core',
        (
            'If the type has values declared for the constraints, they are included as validations. '
            'See the mapping for `confloat` below.'
        )
    ],
    [
        'confloat(gt=1, ge=2, lt=6, le=5, multiple_of=2)',
        'number',
        {'maximum': 5, 'exclusiveMaximum': 6, 'minimum': 2, 'exclusiveMinimum': 1, 'multipleOf': 2},
        'JSON Schema Validation',
        'Any argument not passed to the function (not defined) will not be included in the schema.'
    ],
    [
        'PositiveFloat',
        'number',
        {'exclusiveMinimum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NegativeFloat',
        'number',
        {'exclusiveMaximum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NonNegativeFloat',
        'number',
        {'minimum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'NonPositiveFloat',
        'number',
        {'maximum': 0},
        'JSON Schema Validation',
        ''
    ],
    [
        'ConstrainedDecimal',
        'number',
        '',
        'JSON Schema Core',
        (
            'If the type has values declared for the constraints, they are included as validations. '
            'See the mapping for `condecimal` below.'
        )
    ],
    [
        'condecimal(gt=1, ge=2, lt=6, le=5, multiple_of=2)',
        'number',
        {'maximum': 5, 'exclusiveMaximum': 6, 'minimum': 2, 'exclusiveMinimum': 1, 'multipleOf': 2},
        'JSON Schema Validation',
        'Any argument not passed to the function (not defined) will not be included in the schema.'
    ],
    [
        'BaseModel',
        'object',
        '',
        'JSON Schema Core',
        'All the properties defined will be defined with standard JSON Schema, including submodels.'
    ],
    [
        'Color',
        'string',
        {'format': 'color'},
        'Pydantic standard "format" extension',
        '',
    ],
]

headings = [
    'Python type',
    'JSON Schema Type',
    'Additional JSON Schema',
    'Defined in',
]
set([t[1] for t in table])

{'anyOf',
 'array',
 'boolean',
 'enum',
 'integer',
 'null',
 'number',
 'object',
 'string'}